In [1]:
import torch
import wave
import torchaudio
from sympy.codegen.fnodes import dsign
from transformers import Speech2TextProcessor, Speech2TextForConditionalGeneration
from datasets import load_dataset, load_metric
from transformers import Speech2TextForConditionalGeneration, Wav2Vec2Processor, Speech2TextProcessor, TrainingArguments, Trainer, \
    DataCollatorForSeq2Seq

In [2]:
model_checkpoint = "facebook/wav2vec2-base"

In [3]:
model = Speech2TextForConditionalGeneration.from_pretrained("facebook/wav2vec2-base")

processor = Wav2Vec2Processor.from_pretrained("bond005/wav2vec2-mbart50-ru")

ds_russian = load_dataset("bond005/sberdevices_golos_10h_crowd")

You are using a model of type wav2vec2 to instantiate a model of type speech_to_text. This is not supported for all configurations of models and can yield errors.
/Users/Viacheslav.Suvorov/jub/speech2text/venv/lib/python3.11/site-packages/transformers/configuration_utils.py:381: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of Speech2TextForConditionalGeneration were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['encoder.layers.5.self_attn.k_proj.weight', 'encoder.conv.conv_layers.0.weight', 'decoder.layers.3.self_attn.k_proj.bias', 'encoder.layers.9.fc1.weight', 'encoder.layers.11.self_attn.out_proj.weight', 'decoder.layers.0.encoder_attn.out_proj.weight', 'decoder.layers

In [4]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(ds_russian["train"]))

# print(ds_russian["train"][rand_int]["transcription"])
ipd.Audio(data=np.asarray(ds_russian["train"][rand_int]["audio"]["array"]), autoplay=True, rate=16000)

In [5]:
rand_int = random.randint(0, len(ds_russian["train"]))

print("Target text:", ds_russian["train"][rand_int]["transcription"])
print("Input array shape:", np.asarray(ds_russian["train"][rand_int]["audio"]["array"]).shape)
print("Sampling rate:", ds_russian["train"][rand_int]["audio"]["sampling_rate"])

Target text: бутылка молока простоквашино нормализованное два и пять процента заказать с доставкой
Input array shape: (88000,)
Sampling rate: 16000


In [6]:
from transformers import AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained(model_checkpoint)

/Users/Viacheslav.Suvorov/jub/speech2text/venv/lib/python3.11/site-packages/transformers/configuration_utils.py:381: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [7]:
def extract_all_chars(batch):
  all_text = " ".join(filter(lambda x: x is not None, batch["transcription"]))
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [8]:
vocabs = ds_russian.map(
  extract_all_chars,
  batched=True,
  batch_size=-1,
  keep_in_memory=True, 
  remove_columns=ds_russian.column_names["train"]
)

vocab_list = list(set(vocabs["train"]["vocab"][0]) | set(vocabs["test"]["vocab"][0]))
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
# vocab_dict = {v: k for k, v in enumerate(['s', 'a', 's', 'a', 's', 'a', 's', 'a', 's', 'a', 's', 'a', 's', 'a', 's', 'a', 's', 'a', 's', 'a', 's', 'a', 's', 'a', 'h', 'm', 'n', 'b', ])}

Map:   0%|          | 0/7993 [00:00<?, ? examples/s]

Map:   0%|          | 0/793 [00:00<?, ? examples/s]

Map:   0%|          | 0/9994 [00:00<?, ? examples/s]

In [9]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [10]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

35

In [11]:
import json
with open('vocab.json', 'w', encoding='utf-8') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [12]:
from transformers import AutoConfig

config = AutoConfig.from_pretrained(model_checkpoint)

tokenizer_type = config.model_type if config.tokenizer_class is None else None
config = config if config.tokenizer_class is not None else None
tokenizer_type

'wav2vec2'

In [13]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(
  "./",
  config=config,
  tokenizer_type=tokenizer_type,
  unk_token="[UNK]",
  pad_token="[PAD]",
  word_delimiter_token="|",
)

`use_fast` is set to `True` but the tokenizer class does not have a fast version.  Falling back to the slow version.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Speech2TextTokenizer'. 
The class this function is called from is 'Wav2Vec2CTCTokenizer'.


Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize


In [14]:
from transformers import SpeechEncoderDecoderModel, Wav2Vec2Processor

processor = Wav2Vec2Processor.from_pretrained("bond005/wav2vec2-mbart50-ru")
ds_russian = load_dataset("bond005/sberdevices_golos_10h_crowd")
# ds_russian["train"]["text"] = ds_russian["train"]["transcription"]

In [15]:
# from torch.utils.data import Dataset
# 
# 
# class MyDataset(Dataset):
#     def __init__(self, data):
#         self.data = data
# 
#     def __getitem__(self, idx):
#         item = self.data[idx]
#         item['text'] = item.pop('transcription')
#         return item
# 
#     def __len__(self):
#         return len(self.data)
# 
# # Assuming you already have the train, test, and validation datasets as Torch datasets
# train_dataset = MyDataset(ds_russian['train'])
# test_dataset = MyDataset(test)
# validation_dataset = MyDataset(validation)

In [16]:
for item in train_dataset:
    print(item['text'])

NameError: name 'train_dataset' is not defined

In [17]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched" to ensure mapping is correct
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])
    batch["text"] = batch["transcription"] if batch["transcription"] else ""
    
    with processor.as_target_processor():
        batch["labels"] =  processor(batch["text"]).input_ids
        
    print(batch)
    return batch

In [18]:
ds_russian = ds_russian.map(prepare_dataset, remove_columns=ds_russian.column_names["train"], num_proc=1)

In [19]:
max_input_length_in_sec = 4.0
ds_russian["train"] = ds_russian["train"].filter(lambda x: x < max_input_length_in_sec * processor.feature_extractor.sampling_rate, input_columns=["input_length"])

In [20]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [21]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [22]:
!pip install jiwer
wer_metric = load_metric("wer")


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


/var/folders/hw/cr987xm57j13v444jlc16lw00000gp/T/ipykernel_17837/2209067879.py:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  wer_metric = load_metric("wer")


In [23]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [33]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    model_checkpoint, 
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
)

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'lm_head.weight', 'lm_head.bias', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [34]:
from huggingface_hub import notebook_login

notebook_login()

In [35]:
ds_russian["train"][0]

{'input_values': [-0.0010180305689573288,
  -0.0007876614690758288,
  -0.0007876614690758288,
  -0.0010180305689573288,
  -0.0010180305689573288,
  -0.0010180305689573288,
  -0.0007876614690758288,
  -0.0010180305689573288,
  -0.0005572923691943288,
  -0.0010180305689573288,
  -0.0007876614690758288,
  -0.0007876614690758288,
  -0.0007876614690758288,
  -0.0007876614690758288,
  -0.0007876614690758288,
  -0.0007876614690758288,
  -0.0007876614690758288,
  -0.0010180305689573288,
  -0.0007876614690758288,
  -0.0007876614690758288,
  -0.0007876614690758288,
  -0.0007876614690758288,
  -0.0007876614690758288,
  -0.0005572923691943288,
  -0.0007876614690758288,
  -0.0007876614690758288,
  -0.0007876614690758288,
  -0.0007876614690758288,
  -0.0005572923691943288,
  -0.0005572923691943288,
  -0.0007876614690758288,
  -0.0005572923691943288,
  -0.0005572923691943288,
  -0.0007876614690758288,
  -0.0007876614690758288,
  -0.0007876614690758288,
  -0.0005572923691943288,
  -0.00078766146907582

In [36]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="results",
  group_by_length=True,
  per_device_train_batch_size=32,
  evaluation_strategy="steps",
  num_train_epochs=30,
  fp16=False,
  gradient_checkpointing=True,
  save_steps=500,
  eval_steps=500,
  logging_steps=500,
  learning_rate=1e-4,
  weight_decay=0.005,
  warmup_steps=1000,
  save_total_limit=2,
  push_to_hub=True,
) 

In [37]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=ds_russian["train"],
    eval_dataset=ds_russian["test"],
    tokenizer=processor.feature_extractor,
)

In [38]:
import os
os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'

In [ ]:
trainer.train()